In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from functools import reduce

from IPython.display import display
# get the library
import arivale_data_interface as adi
# apply settings so the lib knows where the data is
adi.apply_isb_config()
#list the local snapshots
# local_snapshots = adi.list_local_snapshots()

# from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt
import seaborn as sns

/opt/conda/envs/arivale-py3/lib/python3.9/site-packages/seaborn/cm.py:1582: MatplotlibDeprecationWarning: The register_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps.register(name)`` instead.
  mpl_cm.register_cmap(_name, _cmap)
/opt/conda/envs/arivale-py3/lib/python3.9/site-packages/seaborn/cm.py:1583: MatplotlibDeprecationWarning: The register_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps.register(name)`` instead.
  mpl_cm.register_cmap(_name + "_r", _cmap_r)


In [2]:
#Drop rows or columns with >20% missingness
def clean_df(df):
    for x in df.columns:
        if (df[x].isnull().sum(axis=0)) > len(df) * 0.2:
            df.drop(columns=[x], inplace=True)
    for x in df.index:
        if df.loc[x, :].isnull().sum() > df.shape[1] * 0.2:
            df.drop(index=x, inplace=True)
    return df

In [3]:
## Client Info


In [45]:
rfd = pd.read_csv("RepeatedFirstDraw.tsv", sep='\t')
#names=['public_client_id', '0', '1', '2', '3', '4', '5', '6', '7', '8']

In [46]:
rfd = rfd[['Unnamed: 0']]

In [47]:
rfd = rfd.rename(columns={"Unnamed: 0": "public_client_id"})

In [48]:
rfd.head()

,public_client_id
0,1074064
1,1149460
2,1262786
3,1271170
4,1287554


In [42]:
nfd = pd.read_csv("NoFirstDraw.tsv", sep='\t')

In [43]:
nfd = nfd[['Unnamed: 0']]
nfd = nfd.rename(columns={"Unnamed: 0": "public_client_id"})

In [44]:
nfd.head()

,public_client_id
0,1048730
1,1073774
2,1133979
3,1298856
4,1486356


In [54]:
fd = rfd.merge(nfd, how='outer', on='public_client_id')

In [56]:
fd.count()

public_client_id    28
dtype: int64

In [61]:
fd['public_client_id'] = fd['public_client_id'].astype('object')

In [71]:
clients = pd.read_csv("/shared-data/snapshots/arivale_snapshot_ISB_2020-03-16_2156/clients.tsv", skiprows=13, sep='\t', converters={'public_client_id': str})
print(clients.shape)
clients.head()

(6133, 17)


,public_client_id,genome_id,genome_vendor,multiple_genome_ids,client_id,user_id,sex,race,has_research_consent,enterprise,coach,region,country,current_program,latest_program,is_helix,age
0,01000552,NaN,NaN,NaN,280707,c9570225-71ed-41d8-960c-3d8222b15f96,M,white,True,d4cd0dabcf4caa22,NaN,ND,NaN,NaN,Non-assay Subscription,False,54
1,01000978,NaN,NaN,NaN,278905,7ace8f46-be13-4e37-85ba-f73cba483f09,F,white,True,d4cd0dabcf4caa22,NaN,AL,NaN,NaN,Non-assay Subscription,False,52
2,01001181,NaN,NaN,NaN,276325,5e1c0585-9224-47a3-ae6d-a4c4b6086e1f,F,black or african-american,True,d4cd0dabcf4caa22,NaN,CA,NaN,NaN,Non-assay Subscription,False,48
3,01001298,DS-309044,WUXI_SNP,NaN,275178,927b5bbb-906a-4069-a268-878eb70fbf14,F,white,True,d4cd0dabcf4caa22,NaN,CA,NaN,NaN,Subscription w/blood draw every 6 months,False,61
4,01001467,DS-303402,WUXI_SNP,NaN,274017,a12f0149-500e-4da4-b1b5-b5d516be4c32,F,black or african-american,True,d4cd0dabcf4caa22,NaN,RI,NaN,NaN,Subscription w/blood draw every 6 months (May ...,False,31


In [72]:
clients = clients[['public_client_id','sex','race','age']]

In [73]:
clients_clean = clean_df(clients)
print(clients_clean.shape)
clients_clean.head()

(6133, 4)


,public_client_id,sex,race,age
0,01000552,M,white,54
1,01000978,F,white,52
2,01001181,F,black or african-american,48
3,01001298,F,white,61
4,01001467,F,black or african-american,31


In [7]:
## Frailty Measures

In [74]:
fr_measures = pd.read_csv("/shared-libs/useful-files/frailty_measures_kanelab/FI_workshop_040124/combination_fi_040124.csv")
print(fr_measures.shape)
fr_measures.head()

(3090, 83)


,Unnamed: 0,public_client_id,self_fi,self_fi_sum,num_na_self,days_in_program,sex,age,race,Dise1,Dise2,Dise3,Dise4,Dise5,Dise6,Dise7,Dise8,Dise9,Dise10,Dise11,Dise12,Dise13,Dise14,Dise15,SAT_1,SAT_2,SAT_3,SAT_4,SAT_5,SAT_6,ACT_1,ACT_2,ACT_3,ACT_4,ACT_5,ACT_6,ACT_7,ACT_8,ACT_9,Med_1,Med_2,Med_3,Diges1,Diges2,CHEM1,CHEM2,CHEM3,CHEM4,CHEM5,CHEM6,CHEM7,CHEM8,CHEM9,CHEM10,CHEM11,CHEM12,CHEM13,CHEM14,CHEM15,CHEM16,CHEM17,CHEM18,CHEM19,CHEM20,CHEM21,CHEM22,CHEM23,CHEM24,CHEM25,CHEM26,CHEM27,CHEM28,CHEM29,BP1,BP2,BP3,BP4,BP5,num_na_lab,lab_fi,lab_fi_sum,num_na_both,merge_fi
0,1,01001548,0.188571,6.6,0,4,M,49,white,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.8,0.8,0.6,0.8,1.0,0.6,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1.0,0,1,0.0,0,1,1,0.0,1,1.0,0.0,0,0,0,0.0,0,0.0,1,1.0,1.0,1.0,1.0,NaN,1.0,0.0,1,0.363636,12,1,0.273529
1,2,01002192,0.114286,4.0,0,0,M,26,white,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.6,0.2,0.6,0.2,0.6,0.8,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0,1,0,0,0,0,0,0,0,0.0,0,1,0.0,0,0,0,0.0,1,1.0,0.0,0,0,0,0.0,0,0.0,0,1.0,1.0,1.0,1.0,NaN,1.0,0.0,1,0.272727,9,1,0.191176
2,3,01003555,0.157576,5.2,2,0,M,47,white,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,0.0,0.0,0.4,0.4,0.4,0.4,0.8,0.8,0.0,0,NaN,0.0,0.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1.0,0,0,1.0,0,1,0,0.0,0,1.0,0.0,0,0,0,1.0,0,0.0,1,1.0,1.0,1.0,1.0,NaN,1.0,0.0,1,0.333333,11,3,0.245455
3,4,01003662,0.162857,5.7,0,1,M,26,white,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.8,0.8,0.4,0.8,0.4,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.5,0,0,0,0,1,0,0,0,0,0.0,0,1,0.0,0,0,0,1.0,1,1.0,0.0,0,0,0,0.0,0,0.0,1,1.0,1.0,0.0,1.0,NaN,0.0,0.0,1,0.272727,9,1,0.216176
4,5,01006695,0.160000,5.6,0,0,M,61,white,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,1.0,0.0,0.2,0.6,0.2,0.8,0.8,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,0,0,0,0.0,0,0.0,0,0.0,1.0,0.0,0.0,NaN,0.0,0.0,1,0.030303,1,1,0.097059


In [75]:
fr_sub = fr_measures[['public_client_id','merge_fi']]

In [76]:
fr_clean = clean_df(fr_sub)
print (fr_clean.shape)
fr_clean.head()

(3090, 2)


,public_client_id,merge_fi
0,01001548,0.273529
1,01002192,0.191176
2,01003555,0.245455
3,01003662,0.216176
4,01006695,0.097059


In [11]:
## Metabolomics

In [77]:
# mets = pd.read_csv("Metabolite_module_eigenvalues.tsv", sep='\t', converters={'public_client_id': str, 'MElightcyan': float, 'MEgreenyellow': float, 'MEpurple': float, 'MEblue': float, 'MEbrown': float, 'MEgreen': float,'MEmidnightblue': float, 'MEcyan': float, 'MEtan': float, 'MEyellow': float, 'MEblack': float, 'MEmagenta': float, 'MEred': float, 'MEsalmon': float, 'MEpink': float, 'MEturquoise': float, 'MEgrey': float})
mets = pd.read_csv("Metabolite_module_eigenvalues.tsv", sep='\t', converters={'public_client_id': str})
print(mets.shape)
mets.head()

(1837, 18)


,public_client_id,MElightcyan,MEgreenyellow,MEpurple,MEblue,MEbrown,MEgreen,MEmidnightblue,MEcyan,MEtan,MEyellow,MEblack,MEmagenta,MEred,MEsalmon,MEpink,MEturquoise,MEgrey
0,01001621,0.009158972401358477,-0.004991957095034002,0.04281506865268347,0.020383699145274832,-0.023159287595536978,-0.0295591176572836,-0.012911943753624243,0.026542515148340152,-0.011651000420624235,0.01020125944698811,-0.03393747003989333,0.013952891149805717,-0.0036457095107202925,-0.05233763181544071,-0.012892465165996594,-0.002270451293143785,0.00663075463377354
1,01003555,-0.03473182306474229,-0.010244901499640463,0.10472546510247226,0.016472057593908154,0.008063999491121146,0.012080351993158506,0.011604113700568576,0.05849519790046305,0.028027282840552233,0.039301217250794926,-0.015507796048394328,0.0061160861115037105,0.037544665410294636,-0.02214257132933654,0.02891098905901219,-0.019802634277396303,-0.007770971577071335
2,01003758,-0.017783483118118778,-0.017407160498238136,-0.03213807539093401,-0.03393164534948164,-0.046343708349061374,0.0016643002527750611,0.04370849057495358,0.019791590782546203,-0.007266084312998367,-0.00823031340985602,-0.011667627231683386,-0.008740350155123634,-0.024078542822676262,-0.006341578355578976,-0.020426668393178502,0.0540469458276216,-0.021360909047924224
3,01004551,0.004941946127868267,-0.0235837656153122,-0.02940195420964963,0.013448721015523607,-0.032274796688545715,-0.014216612011301382,0.02311445281744931,-0.03978256461417802,-0.0377597000889478,0.012188864495833994,-0.03913724452983689,0.012865374473045218,0.00811160770833459,-0.046144762451688075,-0.0294495985837565,-0.03747527998249713,0.05021792367455064
4,01004760,-0.03076164286205698,-0.015196144895858885,0.02850679283143602,0.033701766002735174,-0.013048933268596807,0.04525788180329561,0.07095243840495462,0.05046109772416319,0.0014148328085066126,0.05354461388116871,0.05272778577972037,0.01623077079381199,0.04138653610539182,-0.007396476415490808,0.029927206799272046,0.0023574284884120117,0.05376382586598539


In [78]:
col_names = ['MElightcyan', 'MEgreenyellow', 'MEpurple', 'MEblue', 'MEbrown', 'MEgreen','MEmidnightblue','MEcyan','MEtan','MEyellow','MEblack','MEmagenta','MEred','MEsalmon','MEpink','MEturquoise','MEgrey']

mets[col_names] = mets[col_names].apply(pd.to_numeric, errors='coerce')

In [79]:
print(mets.shape)

(1837, 18)


In [80]:
missing_values = mets.isnull().sum()

# Calculate total number of entries in the DataFrame
total_entries = mets.shape[0] * mets.shape[1]

# Calculate percentage of missing values for each column
percentage_missing_values = (missing_values / total_entries) * 100

print("Percentage of missing values in each column:")
print(percentage_missing_values)

Percentage of missing values in each column:
public_client_id    0.000000
MElightcyan         0.003024
MEgreenyellow       0.003024
MEpurple            0.003024
MEblue              0.003024
MEbrown             0.003024
MEgreen             0.003024
MEmidnightblue      0.003024
MEcyan              0.003024
MEtan               0.003024
MEyellow            0.003024
MEblack             0.003024
MEmagenta           0.003024
MEred               0.003024
MEsalmon            0.003024
MEpink              0.003024
MEturquoise         0.003024
MEgrey              0.003024
dtype: float64


In [81]:
mets_clean = clean_df(mets)
print(mets_clean.shape)
mets_clean.head()

(1836, 18)


,public_client_id,MElightcyan,MEgreenyellow,MEpurple,MEblue,MEbrown,MEgreen,MEmidnightblue,MEcyan,MEtan,MEyellow,MEblack,MEmagenta,MEred,MEsalmon,MEpink,MEturquoise,MEgrey
0,01001621,0.009159,-0.004992,0.042815,0.020384,-0.023159,-0.029559,-0.012912,0.026543,-0.011651,0.010201,-0.033937,0.013953,-0.003646,-0.052338,-0.012892,-0.002270,0.006631
1,01003555,-0.034732,-0.010245,0.104725,0.016472,0.008064,0.012080,0.011604,0.058495,0.028027,0.039301,-0.015508,0.006116,0.037545,-0.022143,0.028911,-0.019803,-0.007771
2,01003758,-0.017783,-0.017407,-0.032138,-0.033932,-0.046344,0.001664,0.043708,0.019792,-0.007266,-0.008230,-0.011668,-0.008740,-0.024079,-0.006342,-0.020427,0.054047,-0.021361
3,01004551,0.004942,-0.023584,-0.029402,0.013449,-0.032275,-0.014217,0.023114,-0.039783,-0.037760,0.012189,-0.039137,0.012865,0.008112,-0.046145,-0.029450,-0.037475,0.050218
4,01004760,-0.030762,-0.015196,0.028507,0.033702,-0.013049,0.045258,0.070952,0.050461,0.001415,0.053545,0.052728,0.016231,0.041387,-0.007396,0.029927,0.002357,0.053764


In [82]:
## Proteomics

In [83]:
prots = pd.read_csv("Protein_module_eigenvalues.tsv", sep='\t', converters={'public_client_id': str, 'MEturquoise': float, 'MEblue': float, 'MEbrown': float, 'MEgrey': float})
print(prots.shape)
prots.head()

(1373, 5)


,public_client_id,MEturquoise,MEblue,MEbrown,MEgrey
0,01001621,-0.018373,-0.003168,-0.007544,0.008927
1,01001661,-0.022060,-0.025751,-0.023461,0.014137
2,01002192,-0.000425,0.048339,0.015641,0.008648
3,01002885,0.013053,-0.017653,-0.010877,-0.006728
4,01003555,0.009569,0.009963,0.004403,0.024474


In [84]:
col_names = ['MEturquoise', 'MEblue', 'MEbrown', 'MEgrey']
new_names = {col: col + '_prots' for col in col_names}
prots = prots.rename(columns=new_names)
print(prots.shape)
prots.head()

(1373, 5)


,public_client_id,MEturquoise_prots,MEblue_prots,MEbrown_prots,MEgrey_prots
0,01001621,-0.018373,-0.003168,-0.007544,0.008927
1,01001661,-0.022060,-0.025751,-0.023461,0.014137
2,01002192,-0.000425,0.048339,0.015641,0.008648
3,01002885,0.013053,-0.017653,-0.010877,-0.006728
4,01003555,0.009569,0.009963,0.004403,0.024474


In [21]:
## Labs

In [85]:
labs = adi.read_snapshot_file("/shared-data/snapshots/arivale_snapshot_ISB_2020-03-16_2156/chemistries.tsv", clean=True, parse_dates=True)
print(labs.shape)
labs.head()

(11167, 140)


,public_client_id,vendor,vendor_observation_id,observation_id,reflexive,fasting,days_in_program,days_since_first_call,days_since_first_draw,month,weekday,season,A/G RATIO,"ADIPONECTIN, SERUM",ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,"ANTIOXID CAP, TOTAL",ARACHIDONIC ACID,"ARSENIC, BLOOD",ASAT (SGOT),BASOPHILS,BASOPHILS ABSOLUTE,"BILIRUBIN, DIRECT","BILIRUBIN, INDIRECT","BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","COPPER, RBC","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EOSINOPHILS,EOSINOPHILS ABSOLUTE,EPA,EPA/AA,FERRITIN,"FOLIC ACID, SERUM","GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,"GLUTATHIONE, TOTAL",GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL LARGE,HDL PARTICLE NUMBER,HDL PARTICLE PERCENTILE,HDL SIZE PERCENTILE,HEMATOCRIT,HEMOGLOBIN,HOMA-IR,"HOMOCYSTEINE, SERUM",IDL PARTICLE NUMBER,IL-6,IL-8,IMMATURE GRANULOCYTES,IMMATURE GRANULOCYTES ABSOLUTE,INSULIN,LACTIC DEHYDROGENASE,LARGE HDL PARTICLE PERCENTILE,LARGE LDL PARTICLE NUMBER,LARGE MED HDL PARTICLE NUMBER,LARGE MED VLDL PARTICLE NUMBER,LARGE VLDL PARTICLE PERCENTILE,LDL MEDIUM,LDL PARTICLE NUMBER,LDL PEAK SIZE,LDL SIZE PERCENTILE,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,"LEAD, BLOOD",LINOLEIC_ACID,LPIR SCORE PERCENTILE,LPIR_SCORE,LP_PLA2,LYMPHOCYTES,LYMPHOCYTES ABSOLUTE,"MAGNESIUM, SERUM","MANGANESE, SERUM",MCH,MCHC,MCV,MEDIUM HDL PARTICLE NUMBER,MEDIUM VLDL PARTICLE NUMBER,"MERCURY, BLOOD",METHYLMALONIC ACID,MONOCYTES,MONOCYTES ABSOLUTE,MPV,"NEUTROPHIL, SEGS",NEUTROPHILS ABSOLUTE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,OXLDL,"PAI-1 ANTIGEN, QNT",PFFA,PHOSPHORUS INORGANIC,PLATELET COUNT,PLATELET COUNT THOUSAND,POTASSIUM,PROT AND PFFA CALC,PROTEIN,"PROTEIN, TOTAL SERUM",QUICKI,RDW,RED CELL COUNT,"SELENIUM, SERUM",SMALL HDL PARTICLE NUMBER,SMALL LDL PARTICLE PERCENTILE,SMALL VLDL PARTICLE NUMBER,SODIUM,SUPEROX DISMUT SOD,TNF-ALPHA,TOTAL NEUTROPHILS,TOTAL NEUTROPHILS AB,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT","VITAMIN D2, 25-OH","VITAMIN D3, 25-OH",VLDL LDL PARTICLE NUMBER,VLDL PARTICLE NUMBER,VLDL SIZE PERCENTILE,VLDL TRIGLYCERIDES,WHITE CELL COUNT,ZINC,"ZINC, RBC",leptin,zinc_plasma_or_serum
0,01000261,Quest,YH289668,10271,False,True,65,-9.0,0,Jan,Tue,winter,2.0,10.4,12.0,4.5,43.0,1895.3,8.0,2.0,15.0,0.3,0.02,0.1,0.4,0.5,18.0,9.4,25.0,101.0,232.0,0.67,0.65,0.20,3.0,NaN,1.9,0.11,0.9,0.1125,63.0,16.3,97.0,NaN,13.0,2.2,90.0,950.0,5.1,67.0,8342.0,NaN,NaN,NaN,45.0,14.5,2.222222,9.1,NaN,1.62,8.75,NaN,NaN,10.0,131.0,NaN,NaN,NaN,NaN,NaN,528.0,1807.0,219.3,NaN,228.0,142.0,NaN,1.2,NaN,NaN,NaN,NaN,16.7,0.98,NaN,0.5,29.7,32.2,92.0,NaN,NaN,2.0,83.0,1.5,0.09,10.1,79.6,4.68,4.0,7.3,NaN,NaN,NaN,108.2,NaN,3.1,244000.0,244.0,4.2,NaN,NaN,6.7,0.147007,13.8,4.90,140.0,NaN,NaN,NaN,136.0,66.6,1.9,79.6,4.68,114.0,1.701493,12.0,4.1,58.0,4.0,58.0,NaN,NaN,NaN,NaN,5.9,980.0,9.8,NaN,NaN
1,01000261,LCA,B0040998784,65656,False,True,198,124.0,133,May,Tue,spring,2.0,12.4,12.0,4.3,40.0,NaN,8.8,NaN,18.0,0.0,0.00,NaN,NaN,0.6,12.0,9.6,22.0,99.0,230.0,NaN,0.77,0.14,3.0,1.1,2.0,0.10,0.9,NaN,69.0,19.9,91.0,105.0,9.0,2.1,90.0,287.0,5.5,68.0,NaN,32.7,63.5,54.4,42.8,14.6,1.222222,9.8,246.0,0.70,11.30,0.0,0.0,5.5,NaN,86.6,867.0,16.5,36.2,24.9,NaN,1742.0,NaN,90.6,447.0,141.0,21.4,1.0,24.1,22.5,25.0,174.0,24.0,1.20,1.6,NaN,30.4,34.1,89.0,6.1,37.0,2.8,115.0,4.0,0.20,NaN,NaN,NaN,5.0,7.2,5.0,36.0,42.0,NaN,93.0,NaN,NaN,247.0,4.2,626.0,533.0,6.4,NaN,12.9,4.81,NaN,16.0,42.0,10.0,138.0,NaN,0.9,70.0,3.40,107.0,1.573529,9.0,4.2,55.3,NaN,NaN,1753.0,45.5,48.9,69.1,4.9,1410.0,NaN,NaN,NaN
2,01001298,LCA,AV27365801,2837393,False,True,7,0.0,0,Sep,Tue,fall,1.8,NaN,25.0,4.2,63.0,NaN,11.7,NaN,28.0,NaN,NaN,NaN,NaN,0.3,18.0,9.0,24.0,101.0,271.0,NaN,0.68,0.60,3.0,1.2,NaN,NaN,1.2,NaN,57.0,NaN,95.0,109.0,15.0,2.4,94.0,NaN,5.6,64.0,NaN,35.7,NaN,NaN,NaN,NaN,1.601481,7.5,NaN,NaN,NaN,NaN,NaN,6.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2307.0,NaN,NaN,893.0,187.0,21.5,NaN,25.0,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [86]:
# Get lab data at baseline
labs_baseline = labs[labs.days_since_first_draw <= 60]
labs_baseline.head()

,public_client_id,vendor,vendor_observation_id,observation_id,reflexive,fasting,days_in_program,days_since_first_call,days_since_first_draw,month,weekday,season,A/G RATIO,"ADIPONECTIN, SERUM",ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,"ANTIOXID CAP, TOTAL",ARACHIDONIC ACID,"ARSENIC, BLOOD",ASAT (SGOT),BASOPHILS,BASOPHILS ABSOLUTE,"BILIRUBIN, DIRECT","BILIRUBIN, INDIRECT","BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","COPPER, RBC","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EOSINOPHILS,EOSINOPHILS ABSOLUTE,EPA,EPA/AA,FERRITIN,"FOLIC ACID, SERUM","GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,"GLUTATHIONE, TOTAL",GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL LARGE,HDL PARTICLE NUMBER,HDL PARTICLE PERCENTILE,HDL SIZE PERCENTILE,HEMATOCRIT,HEMOGLOBIN,HOMA-IR,"HOMOCYSTEINE, SERUM",IDL PARTICLE NUMBER,IL-6,IL-8,IMMATURE GRANULOCYTES,IMMATURE GRANULOCYTES ABSOLUTE,INSULIN,LACTIC DEHYDROGENASE,LARGE HDL PARTICLE PERCENTILE,LARGE LDL PARTICLE NUMBER,LARGE MED HDL PARTICLE NUMBER,LARGE MED VLDL PARTICLE NUMBER,LARGE VLDL PARTICLE PERCENTILE,LDL MEDIUM,LDL PARTICLE NUMBER,LDL PEAK SIZE,LDL SIZE PERCENTILE,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,"LEAD, BLOOD",LINOLEIC_ACID,LPIR SCORE PERCENTILE,LPIR_SCORE,LP_PLA2,LYMPHOCYTES,LYMPHOCYTES ABSOLUTE,"MAGNESIUM, SERUM","MANGANESE, SERUM",MCH,MCHC,MCV,MEDIUM HDL PARTICLE NUMBER,MEDIUM VLDL PARTICLE NUMBER,"MERCURY, BLOOD",METHYLMALONIC ACID,MONOCYTES,MONOCYTES ABSOLUTE,MPV,"NEUTROPHIL, SEGS",NEUTROPHILS ABSOLUTE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,OXLDL,"PAI-1 ANTIGEN, QNT",PFFA,PHOSPHORUS INORGANIC,PLATELET COUNT,PLATELET COUNT THOUSAND,POTASSIUM,PROT AND PFFA CALC,PROTEIN,"PROTEIN, TOTAL SERUM",QUICKI,RDW,RED CELL COUNT,"SELENIUM, SERUM",SMALL HDL PARTICLE NUMBER,SMALL LDL PARTICLE PERCENTILE,SMALL VLDL PARTICLE NUMBER,SODIUM,SUPEROX DISMUT SOD,TNF-ALPHA,TOTAL NEUTROPHILS,TOTAL NEUTROPHILS AB,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT","VITAMIN D2, 25-OH","VITAMIN D3, 25-OH",VLDL LDL PARTICLE NUMBER,VLDL PARTICLE NUMBER,VLDL SIZE PERCENTILE,VLDL TRIGLYCERIDES,WHITE CELL COUNT,ZINC,"ZINC, RBC",leptin,zinc_plasma_or_serum
0,01000261,Quest,YH289668,10271,False,True,65,-9.0,0,Jan,Tue,winter,2.0,10.4,12.0,4.5,43.0,1895.3,8.0,2.0,15.0,0.3,0.02,0.1,0.4,0.5,18.0,9.4,25.0,101.0,232.0,0.67,0.65,0.20,3.0,NaN,1.9,0.11,0.9,0.1125,63.0,16.3,97.0,NaN,13.0,2.2,90.0,950.0,5.1,67.0,8342.0,NaN,NaN,NaN,45.0,14.5,2.222222,9.1,NaN,1.62,8.75,NaN,NaN,10.0,131.0,NaN,NaN,NaN,NaN,NaN,528.0,1807.0,219.3,NaN,228.0,142.0,NaN,1.2,NaN,NaN,NaN,NaN,16.7,0.98,NaN,0.5,29.7,32.2,92.0,NaN,NaN,2.0,83.0,1.5,0.09,10.1,79.6,4.68,4.0,7.3,NaN,NaN,NaN,108.2,NaN,3.1,244000.0,244.0,4.2,NaN,NaN,6.7,0.147007,13.8,4.90,140.0,NaN,NaN,NaN,136.0,66.6,1.9,79.6,4.68,114.0,1.701493,12.0,4.1,58.0,4.0,58.0,NaN,NaN,NaN,NaN,5.9,980.0,9.8,NaN,NaN
2,01001298,LCA,AV27365801,2837393,False,True,7,0.0,0,Sep,Tue,fall,1.8,NaN,25.0,4.2,63.0,NaN,11.7,NaN,28.0,NaN,NaN,NaN,NaN,0.3,18.0,9.0,24.0,101.0,271.0,NaN,0.68,0.60,3.0,1.2,NaN,NaN,1.2,NaN,57.0,NaN,95.0,109.0,15.0,2.4,94.0,NaN,5.6,64.0,NaN,35.7,NaN,NaN,NaN,NaN,1.601481,7.5,NaN,NaN,NaN,NaN,NaN,6.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2307.0,NaN,NaN,893.0,187.0,21.5,NaN,25.0,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4,7.4,5.4,39.7,NaN,NaN,NaN,NaN,NaN,NaN,4.1,NaN,NaN,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,100.0,1.562500,12.0,4.1,41.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01001467,LCA,AV29176623,2500980,False,True,15,1.0,0,Jun,Wed,summer,1.3,NaN,24.0,4.0,55.0,NaN,11.8,NaN,19.0,NaN,NaN,NaN,NaN,0.3,11.0,9.4,23.0,98.0,211.0,NaN,0.80,0.56,2.7,0.7,NaN,NaN,0.4,NaN,30.0,NaN,99.0,114.0,46.0,3.0,82.0,NaN,5.2,71.0,NaN,36.3,NaN,NaN,NaN,NaN,0.141728,7.2,NaN,NaN,NaN,NaN,NaN,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1263.0,NaN,NaN,247.0,117.0,21.4,NaN,25.5,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.8,10.8,3.8,41.2,NaN,NaN,NaN,Na

In [87]:
labs_fast_values = labs_baseline['fasting'].value_counts()
print(labs_fast_values)

fasting
True     4791
False     102
Name: count, dtype: int64


In [88]:
labs_fast = labs_baseline[labs_baseline['fasting'] == True]

In [89]:
labs_fast_values = labs_fast['fasting'].value_counts()
print(labs_fast_values)

fasting
True    4791
Name: count, dtype: int64


In [90]:
labs_drop = labs_fast[labs_fast.vendor == 'LCA'].drop(['vendor_observation_id','fasting','observation_id','days_since_first_call','days_in_program',
                                                                     'days_since_first_draw','month','weekday', 'season', 'vendor','reflexive'],
                                                                   axis=1)
print(labs_drop.shape)
labs_drop.head()

(4203, 129)


,public_client_id,A/G RATIO,"ADIPONECTIN, SERUM",ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,"ANTIOXID CAP, TOTAL",ARACHIDONIC ACID,"ARSENIC, BLOOD",ASAT (SGOT),BASOPHILS,BASOPHILS ABSOLUTE,"BILIRUBIN, DIRECT","BILIRUBIN, INDIRECT","BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","COPPER, RBC","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EOSINOPHILS,EOSINOPHILS ABSOLUTE,EPA,EPA/AA,FERRITIN,"FOLIC ACID, SERUM","GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,"GLUTATHIONE, TOTAL",GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL LARGE,HDL PARTICLE NUMBER,HDL PARTICLE PERCENTILE,HDL SIZE PERCENTILE,HEMATOCRIT,HEMOGLOBIN,HOMA-IR,"HOMOCYSTEINE, SERUM",IDL PARTICLE NUMBER,IL-6,IL-8,IMMATURE GRANULOCYTES,IMMATURE GRANULOCYTES ABSOLUTE,INSULIN,LACTIC DEHYDROGENASE,LARGE HDL PARTICLE PERCENTILE,LARGE LDL PARTICLE NUMBER,LARGE MED HDL PARTICLE NUMBER,LARGE MED VLDL PARTICLE NUMBER,LARGE VLDL PARTICLE PERCENTILE,LDL MEDIUM,LDL PARTICLE NUMBER,LDL PEAK SIZE,LDL SIZE PERCENTILE,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,"LEAD, BLOOD",LINOLEIC_ACID,LPIR SCORE PERCENTILE,LPIR_SCORE,LP_PLA2,LYMPHOCYTES,LYMPHOCYTES ABSOLUTE,"MAGNESIUM, SERUM","MANGANESE, SERUM",MCH,MCHC,MCV,MEDIUM HDL PARTICLE NUMBER,MEDIUM VLDL PARTICLE NUMBER,"MERCURY, BLOOD",METHYLMALONIC ACID,MONOCYTES,MONOCYTES ABSOLUTE,MPV,"NEUTROPHIL, SEGS",NEUTROPHILS ABSOLUTE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,OXLDL,"PAI-1 ANTIGEN, QNT",PFFA,PHOSPHORUS INORGANIC,PLATELET COUNT,PLATELET COUNT THOUSAND,POTASSIUM,PROT AND PFFA CALC,PROTEIN,"PROTEIN, TOTAL SERUM",QUICKI,RDW,RED CELL COUNT,"SELENIUM, SERUM",SMALL HDL PARTICLE NUMBER,SMALL LDL PARTICLE PERCENTILE,SMALL VLDL PARTICLE NUMBER,SODIUM,SUPEROX DISMUT SOD,TNF-ALPHA,TOTAL NEUTROPHILS,TOTAL NEUTROPHILS AB,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT","VITAMIN D2, 25-OH","VITAMIN D3, 25-OH",VLDL LDL PARTICLE NUMBER,VLDL PARTICLE NUMBER,VLDL SIZE PERCENTILE,VLDL TRIGLYCERIDES,WHITE CELL COUNT,ZINC,"ZINC, RBC",leptin,zinc_plasma_or_serum
2,01001298,1.8,NaN,25.0,4.2,63.0,NaN,11.7,NaN,28.0,NaN,NaN,NaN,NaN,0.3,18.0,9.0,24.0,101.0,271.0,NaN,0.68,0.60,3.0,1.2,NaN,NaN,1.2,NaN,57.0,NaN,95.0,109.0,15.0,2.4,94.0,NaN,5.6,64.0,NaN,35.7,NaN,NaN,NaN,NaN,1.601481,7.5,NaN,NaN,NaN,NaN,NaN,6.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2307.0,NaN,NaN,893.0,187.0,21.5,NaN,25.0,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4,7.4,5.4,39.7,NaN,NaN,NaN,NaN,NaN,NaN,4.1,NaN,NaN,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,100.0,1.562500,12.0,4.1,41.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01001467,1.3,NaN,24.0,4.0,55.0,NaN,11.8,NaN,19.0,NaN,NaN,NaN,NaN,0.3,11.0,9.4,23.0,98.0,211.0,NaN,0.80,0.56,2.7,0.7,NaN,NaN,0.4,NaN,30.0,NaN,99.0,114.0,46.0,3.0,82.0,NaN,5.2,71.0,NaN,36.3,NaN,NaN,NaN,NaN,0.141728,7.2,NaN,NaN,NaN,NaN,NaN,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1263.0,NaN,NaN,247.0,117.0,21.4,NaN,25.5,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.8,10.8,3.8,41.2,NaN,NaN,NaN,NaN,NaN,NaN,4.4,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.0,NaN,NaN,NaN,NaN,114.0,1.605634,9.0,2.8,41.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,01001548,1.9,NaN,27.0,4.5,57.0,NaN,10.1,NaN,21.0,NaN,NaN,NaN,NaN,0.6,18.0,9.4,25.0,102.0,205.0,NaN,0.88,1.40,2.0,1.1,NaN,NaN,0.7,NaN,173.0,NaN,101.0,117.0,26.0,2.4,106.0,NaN,5.8,45.0,NaN,NaN,NaN,NaN,NaN,NaN,3.507160,12.5,NaN,NaN,NaN,NaN,NaN,13.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135.0,NaN,NaN,25.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.8,10.3,3.8,39.1,NaN,NaN,NaN,NaN,NaN,NaN,4.1,NaN,NaN,6.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,123.0,2.733333,16.0,6.5,25.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,01001621,1.8,11.8,6.0,3.9,67.0,NaN,11.2,NaN,9.0,1.0,0.1,NaN,NaN,0.2,20.0,9.2,24.0,102.0,189.0,NaN,0.97,1.92,2.2,1.3,3.0,0.2,0.6,NaN,68.0,5.2,66.0,77.0,8.0,2.2,87.0,NaN,5.6,62.0,NaN,32.0,NaN,NaN,40.1,13.7,0.859259,11.6,

In [91]:
labs_clean = clean_df(labs_drop)
print(labs_clean.shape)
labs_clean.head()

(4166, 48)


,public_client_id,A/G RATIO,ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,ARACHIDONIC ACID,ASAT (SGOT),"BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EPA,FERRITIN,"GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL PARTICLE NUMBER,HOMA-IR,"HOMOCYSTEINE, SERUM",INSULIN,LDL PARTICLE NUMBER,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,LINOLEIC_ACID,LPIR_SCORE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,POTASSIUM,"PROTEIN, TOTAL SERUM",SODIUM,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT"
2,01001298,1.8,25.0,4.2,63.0,11.7,28.0,0.3,18.0,9.0,24.0,101.0,271.0,0.68,0.60,3.0,1.2,1.2,57.0,95.0,109.0,15.0,2.4,94.0,5.6,64.0,35.7,1.601481,7.5,6.9,2307.0,893.0,187.0,21.5,25.0,25.0,5.4,7.4,5.4,39.7,4.1,6.6,140.0,100.0,1.562500,12.0,4.1,41.7
3,01001467,1.3,24.0,4.0,55.0,11.8,19.0,0.3,11.0,9.4,23.0,98.0,211.0,0.80,0.56,2.7,0.7,0.4,30.0,99.0,114.0,46.0,3.0,82.0,5.2,71.0,36.3,0.141728,7.2,0.7,1263.0,247.0,117.0,21.4,25.5,25.0,3.8,10.8,3.8,41.2,4.4,7.0,137.0,114.0,1.605634,9.0,2.8,41.2
5,01001548,1.9,27.0,4.5,57.0,10.1,21.0,0.6,18.0,9.4,25.0,102.0,205.0,0.88,1.40,2.0,1.1,0.7,173.0,101.0,117.0,26.0,2.4,106.0,5.8,45.0,NaN,3.507160,12.5,13.4,NaN,NaN,135.0,NaN,25.1,NaN,3.8,10.3,3.8,39.1,4.1,6.9,140.0,123.0,2.733333,16.0,6.5,25.6
6,01001621,1.8,6.0,3.9,67.0,11.2,9.0,0.2,20.0,9.2,24.0,102.0,189.0,0.97,1.92,2.2,1.3,0.6,68.0,66.0,77.0,8.0,2.2,87.0,5.6,62.0,32.0,0.859259,11.6,4.0,1392.0,343.0,114.0,21.6,29.6,25.0,4.1,10.8,4.1,44.2,4.0,6.1,142.0,63.0,1.016129,19.0,6.1,30.5
8,01001661,1.9,10.0,4.6,36.0,10.8,21.0,0.7,15.0,9.3,24.0,100.0,187.0,0.66,1.08,3.0,0.7,1.0,66.0,119.0,137.0,11.0,2.4,89.0,5.6,87.0,44.0,2.439259,8.3,11.1,765.0,90.0,84.0,21.2,27.6,25.0,4.7,9.0,4.7,42.1,4.2,7.0,140.0,78.0,0.896552,10.0,4.2,19.0


In [92]:
dfs_to_merge = [clients_clean, fr_clean, mets_clean, prots, labs_clean]

merged_df = reduce(lambda left, right: pd.merge(left, right, on='public_client_id'), dfs_to_merge)
merged_df = merged_df.drop_duplicates(subset=['public_client_id'])

print(merged_df.shape)
merged_df.head()

(769, 73)


,public_client_id,sex,race,age,merge_fi,MElightcyan,MEgreenyellow,MEpurple,MEblue,MEbrown,MEgreen,MEmidnightblue,MEcyan,MEtan,MEyellow,MEblack,MEmagenta,MEred,MEsalmon,MEpink,MEturquoise,MEgrey,MEturquoise_prots,MEblue_prots,MEbrown_prots,MEgrey_prots,A/G RATIO,ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,ARACHIDONIC ACID,ASAT (SGOT),"BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EPA,FERRITIN,"GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL PARTICLE NUMBER,HOMA-IR,"HOMOCYSTEINE, SERUM",INSULIN,LDL PARTICLE NUMBER,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,LINOLEIC_ACID,LPIR_SCORE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,POTASSIUM,"PROTEIN, TOTAL SERUM",SODIUM,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT"
0,01001621,F,white,54,0.125373,0.009159,-0.004992,0.042815,0.020384,-0.023159,-0.029559,-0.012912,0.026543,-0.011651,0.010201,-0.033937,0.013953,-0.003646,-0.052338,-0.012892,-0.002270,0.006631,-0.018373,-0.003168,-0.007544,0.008927,1.8,6.0,3.9,67.0,11.2,9.0,0.2,20.0,9.2,24.0,102.0,189.0,0.97,1.92,2.2,1.3,0.6,68.0,66.0,77.0,8.0,2.2,87.0,5.6,62.0,32.0,0.859259,11.6,4.0,1392.0,343.0,114.0,21.6,29.6,25.0,4.1,10.8,4.1,44.2,4.0,6.1,142.0,63.0,1.016129,19.0,6.1,30.5
2,01003555,M,white,47,0.245455,-0.034732,-0.010245,0.104725,0.016472,0.008064,0.012080,0.011604,0.058495,0.028027,0.039301,-0.015508,0.006116,0.037545,-0.022143,0.028911,-0.019803,-0.007771,0.009569,0.009963,0.004403,0.024474,2.0,38.0,4.5,78.0,10.0,25.0,0.3,12.0,9.4,22.0,101.0,257.0,1.07,0.93,1.8,1.0,0.5,28.0,82.0,95.0,31.0,2.2,102.0,5.3,47.0,32.3,2.392593,10.0,9.5,2484.0,1506.0,175.0,20.0,29.5,70.0,3.3,13.1,3.3,43.2,4.6,6.7,140.0,176.0,3.744681,13.0,4.7,29.3
4,01003758,F,white,33,0.096517,-0.017783,-0.017407,-0.032138,-0.033932,-0.046344,0.001664,0.043708,0.019792,-0.007266,-0.008230,-0.011668,-0.008740,-0.024079,-0.006342,-0.020427,0.054047,-0.021361,0.008914,0.003977,-0.011993,0.028680,1.8,14.0,4.4,49.0,10.2,18.0,0.4,17.0,9.5,23.0,99.0,159.0,0.71,1.01,2.4,0.7,0.3,53.0,113.0,130.0,19.0,2.4,77.0,4.8,87.0,47.7,1.064691,7.5,5.6,634.0,90.0,59.0,21.0,24.5,26.0,3.4,11.6,3.4,39.6,4.0,6.8,139.0,63.0,0.724138,12.0,4.4,40.8
6,01004551,F,white,35,0.134826,0.004942,-0.023584,-0.029402,0.013449,-0.032275,-0.014217,0.023114,-0.039783,-0.037760,0.012189,-0.039137,0.012865,0.008112,-0.046145,-0.029450,-0.037475,0.050218,-0.032174,0.022518,-0.007890,-0.026328,1.6,17.0,4.0,99.0,11.5,19.0,0.4,15.0,8.5,24.0,101.0,198.0,0.74,0.18,1.9,1.0,0.4,41.0,105.0,121.0,6.0,2.5,79.0,5.2,51.0,31.2,1.131358,13.9,5.8,1409.0,337.0,130.0,21.7,28.4,25.0,3.3,13.1,3.3,43.3,4.2,6.5,139.0,83.0,1.627451,11.0,5.5,17.8
8,01006225,F,white,38,0.259701,0.032993,-0.006151,-0.006772,-0.003167,0.038353,-0.013992,0.019927,-0.010859,0.008608,-0.012860,-0.036614,-0.027978,-0.002400,-0.041678,-0.046364,-0.011555,0.049733,0.012178,-0.010986,0.010039,-0.024647,1.8,22.0,4.4,61.0,11.9,18.0,0.3,16.0,9.3,20.0,103.0,221.0,0.75,1.27,1.5,1.1,0.6,29.0,101.0,117.0,19.0,2.4,97.0,5.4,53.0,40.3,2.514815,7.6,10.5,1757.0,606.0,149.0,21.1,25.5,67.0,3.2,12.9,3.2,41.4,4.3,6.8,140.0,93.0,1.754717,12.0,4.5,19.2


In [93]:
cat_cols = merged_df.select_dtypes(include='object').columns.tolist()
print(cat_cols)

['public_client_id', 'sex', 'race']


In [94]:
sex_cats = merged_df['sex'].value_counts()
print(sex_cats)

sex
F    513
M    256
Name: count, dtype: int64


In [95]:
merged_df['sex'].replace('F', 0,inplace=True)
merged_df['sex'].replace('M', 1,inplace=True)

/tmp/ipykernel_1926254/1472447010.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['sex'].replace('F', 0,inplace=True)
/tmp/ipykernel_1926254/1472447010.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

In [96]:
race_cats = merged_df['race'].value_counts()
print(race_cats)

race
white                                        585
hispanic latino or spanish origin             59
south asian                                   29
black or african-american                     24
east asian                                    23
other                                         19
ashkenazi jewish                              11
middle eastern or north african                8
asian                                          5
native hawaiian or other pacific islander      4
american indian or alaska native               1
afro-caribbean                                 1
Name: count, dtype: int64


In [97]:
low_count_categories = race_cats[race_cats <= 20].index.tolist()
print(low_count_categories)

['other', 'ashkenazi jewish', 'middle eastern or north african', 'asian', 'native hawaiian or other pacific islander', 'american indian or alaska native', 'afro-caribbean']


In [98]:
merged_df.loc[merged_df['race'].isin(low_count_categories), 'race'] = 'other'
race_cats = merged_df['race'].value_counts()
print(race_cats)

race
white                                585
hispanic latino or spanish origin     59
other                                 49
south asian                           29
black or african-american             24
east asian                            23
Name: count, dtype: int64


In [99]:
merged_df = pd.get_dummies(merged_df, columns=['race'])

In [100]:
print(merged_df.shape)
merged_df.head()

(769, 78)


,public_client_id,sex,age,merge_fi,MElightcyan,MEgreenyellow,MEpurple,MEblue,MEbrown,MEgreen,MEmidnightblue,MEcyan,MEtan,MEyellow,MEblack,MEmagenta,MEred,MEsalmon,MEpink,MEturquoise,MEgrey,MEturquoise_prots,MEblue_prots,MEbrown_prots,MEgrey_prots,A/G RATIO,ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,ARACHIDONIC ACID,ASAT (SGOT),"BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EPA,FERRITIN,"GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL PARTICLE NUMBER,HOMA-IR,"HOMOCYSTEINE, SERUM",INSULIN,LDL PARTICLE NUMBER,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,LINOLEIC_ACID,LPIR_SCORE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,POTASSIUM,"PROTEIN, TOTAL SERUM",SODIUM,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT",race_black or african-american,race_east asian,race_hispanic latino or spanish origin,race_other,race_south asian,race_white
0,01001621,0,54,0.125373,0.009159,-0.004992,0.042815,0.020384,-0.023159,-0.029559,-0.012912,0.026543,-0.011651,0.010201,-0.033937,0.013953,-0.003646,-0.052338,-0.012892,-0.002270,0.006631,-0.018373,-0.003168,-0.007544,0.008927,1.8,6.0,3.9,67.0,11.2,9.0,0.2,20.0,9.2,24.0,102.0,189.0,0.97,1.92,2.2,1.3,0.6,68.0,66.0,77.0,8.0,2.2,87.0,5.6,62.0,32.0,0.859259,11.6,4.0,1392.0,343.0,114.0,21.6,29.6,25.0,4.1,10.8,4.1,44.2,4.0,6.1,142.0,63.0,1.016129,19.0,6.1,30.5,False,False,False,False,False,True
2,01003555,1,47,0.245455,-0.034732,-0.010245,0.104725,0.016472,0.008064,0.012080,0.011604,0.058495,0.028027,0.039301,-0.015508,0.006116,0.037545,-0.022143,0.028911,-0.019803,-0.007771,0.009569,0.009963,0.004403,0.024474,2.0,38.0,4.5,78.0,10.0,25.0,0.3,12.0,9.4,22.0,101.0,257.0,1.07,0.93,1.8,1.0,0.5,28.0,82.0,95.0,31.0,2.2,102.0,5.3,47.0,32.3,2.392593,10.0,9.5,2484.0,1506.0,175.0,20.0,29.5,70.0,3.3,13.1,3.3,43.2,4.6,6.7,140.0,176.0,3.744681,13.0,4.7,29.3,False,False,False,False,False,True
4,01003758,0,33,0.096517,-0.017783,-0.017407,-0.032138,-0.033932,-0.046344,0.001664,0.043708,0.019792,-0.007266,-0.008230,-0.011668,-0.008740,-0.024079,-0.006342,-0.020427,0.054047,-0.021361,0.008914,0.003977,-0.011993,0.028680,1.8,14.0,4.4,49.0,10.2,18.0,0.4,17.0,9.5,23.0,99.0,159.0,0.71,1.01,2.4,0.7,0.3,53.0,113.0,130.0,19.0,2.4,77.0,4.8,87.0,47.7,1.064691,7.5,5.6,634.0,90.0,59.0,21.0,24.5,26.0,3.4,11.6,3.4,39.6,4.0,6.8,139.0,63.0,0.724138,12.0,4.4,40.8,False,False,False,False,False,True
6,01004551,0,35,0.134826,0.004942,-0.023584,-0.029402,0.013449,-0.032275,-0.014217,0.023114,-0.039783,-0.037760,0.012189,-0.039137,0.012865,0.008112,-0.046145,-0.029450,-0.037475,0.050218,-0.032174,0.022518,-0.007890,-0.026328,1.6,17.0,4.0,99.0,11.5,19.0,0.4,15.0,8.5,24.0,101.0,198.0,0.74,0.18,1.9,1.0,0.4,41.0,105.0,121.0,6.0,2.5,79.0,5.2,51.0,31.2,1.131358,13.9,5.8,1409.0,337.0,130.0,21.7,28.4,25.0,3.3,13.1,3.3,43.3,4.2,6.5,139.0,83.0,1.627451,11.0,5.5,17.8,False,False,False,False,False,True
8,01006225,0,38,0.259701,0.032993,-0.006151,-0.006772,-0.003167,0.038353,-0.013992,0.019927,-0.010859,0.008608,-0.012860,-0.036614,-0.027978,-0.002400,-0.041678,-0.046364,-0.011555,0.049733,0.012178,-0.010986,0.010039,-0.024647,1.8,22.0,4.4,61.0,11.9,18.0,0.3,16.0,9.3,20.0,103.0,221.0,0.75,1.27,1.5,1.1,0.6,29.0,101.0,117.0,19.0,2.4,97.0,5.4,53.0,40.3,2.514815,7.6,10.5,1757.0,606.0,149.0,21.1,25.5,67.0,3.2,12.9,3.2,41.4,4.3,6.8,140.0,93.0,1.754717,12.0,4.5,19.2,False,False,False,False,False,True


In [106]:
test = merged_df[merged_df.public_client_id == '1074064']

In [107]:
test.head()

,public_client_id,sex,age,merge_fi,MElightcyan,MEgreenyellow,MEpurple,MEblue,MEbrown,MEgreen,MEmidnightblue,MEcyan,MEtan,MEyellow,MEblack,MEmagenta,MEred,MEsalmon,MEpink,MEturquoise,MEgrey,MEturquoise_prots,MEblue_prots,MEbrown_prots,MEgrey_prots,A/G RATIO,ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,ARACHIDONIC ACID,ASAT (SGOT),"BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EPA,FERRITIN,"GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL PARTICLE NUMBER,HOMA-IR,"HOMOCYSTEINE, SERUM",INSULIN,LDL PARTICLE NUMBER,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,LINOLEIC_ACID,LPIR_SCORE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,POTASSIUM,"PROTEIN, TOTAL SERUM",SODIUM,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT",race_black or african-american,race_east asian,race_hispanic latino or spanish origin,race_other,race_south asian,race_white


In [103]:
fd.head()

,public_client_id
0,1048730
1,1073774
2,1074064
3,1133979
4,1149460


In [108]:
clients_removed = merged_df.merge(fd, how='left', on='public_client_id')
print(clients_removed.shape)

(769, 78)


In [84]:
merged_clean.to_csv('AR_cleaned_ISB_aging_new_merge_fi_wgcna.csv', index=False)

In [85]:
df = pd.read_csv("AR_cleaned_ISB_aging_new_merge_fi_wgcna.csv")
print(df.shape)

(769, 78)
